In [1]:
import csv
import os.path
import sys
from lib.general import get_protocol
from lib.compartments import GetParentCompartments
from lib.compartments import GetChildCompartments
from lib.securitylists import prepare_csv_record
from lib.securitylists import export_security_list_rules_to_csv
# from lib.securitylists import export_egress_security_list_rules
# from lib.securitylists import export_ingress_security_list_rules
from lib.securitylists import GetNetworkSecurityList
from lib.vcns import GetVirtualCloudNetworks
from oci.config import from_file
from oci.identity import IdentityClient
from oci.core import VirtualNetworkClient

In [2]:
parent_compartment_name = "admin_comp"
child_compartment_name = "auto_comp"
virtual_cloud_network_name = "auto_vcn"
security_list_name = "auto_sec"
region = "us-ashburn-1"
option = "--csv"
security_rule_export_file = security_list_name + "_rules.csv"


In [3]:
config = from_file() # gets ~./.oci/config and reads to the object
config["region"] = region # Must set the cloud region
identity_client = IdentityClient(config) # builds the identity client method, required to manage compartments
network_client = VirtualNetworkClient(config) # builds the network client method, required to manage network resources

In [4]:
parent_compartments = GetParentCompartments(parent_compartment_name, config, identity_client)
parent_compartments.populate_compartments()
parent_compartment = parent_compartments.return_parent_compartment()

In [5]:
child_compartments = GetChildCompartments(
    parent_compartment.id,
    child_compartment_name,
    identity_client)
child_compartments.populate_compartments()
child_compartment = child_compartments.return_child_compartment()


In [6]:
virtual_cloud_networks = GetVirtualCloudNetworks(
    network_client,
    child_compartment.id,
    virtual_cloud_network_name)
virtual_cloud_networks.populate_virtual_cloud_networks()
virtual_cloud_network = virtual_cloud_networks.return_virtual_cloud_network()


In [7]:
security_lists = GetNetworkSecurityList(
    network_client,
    child_compartment.id,
    virtual_cloud_network.id,
    security_list_name
)
security_lists.populate_security_lists()
security_list = security_lists.return_security_list()

In [8]:
ingress_rules = security_list.ingress_security_rules

In [9]:
egress_rules = security_list.egress_security_rules

In [10]:
prepare_csv_record(egress_rules[2])

{'description': 'Sample rule permits all ICMP traffic to tenancy of type 0 with message code 1 to a subnet',
 'icmp_options': {'code': 1, 'type': 0},
 'is_stateless': False,
 'protocol': 'ICMP',
 'destination': '172.16.0.0/24',
 'destination_type': 'CIDR_BLOCK',
 'source': None,
 'source_type': '',
 'tcp_options': {'destination_port_range': {'max': '', 'min': ''},
  'source_port_range': {'max': '', 'min': ''}},
 'udp_options': {'destination_port_range': {'max': '', 'min': ''},
  'source_port_range': {'max': '', 'min': ''}}}

In [11]:
security_rules = []
for rule in ingress_rules:
    security_rules.append(rule)
for rule in egress_rules:
    security_rules.append(rule)

In [12]:
export_security_list_rules_to_csv(
    security_rule_export_file,
    security_rules,
    ";")